In [1188]:
import boto3
import pprint
from boto3.dynamodb.conditions import Key
import datetime
import dateutil

In [1189]:
# Assumes external process:
#   java -Djava.library.path=./DynamoDBLocal_lib -jar DynamoDBLocal.jar -sharedDb --inMemory

ddb = boto3.resource('dynamodb', endpoint_url='http://localhost:8000')
#dc = boto3.client('dynamodb', endpoint_url='http://localhost:8000')

In [1190]:
try:
    _dbn
except:
    _dbn = 0
_dbn += 1

class DynamoDbDriver(object):
    def __init__(self, ddb):
        self._ddb = ddb
        
        global _dbn
        
        self.name = 'DDB-MEM-%s' % _dbn
        
        entityTableName = 'entities_%s' % _dbn
        mapTableName    = 'map_%s' % _dbn
        
        _dbn += 1
        
        self._entities = ddb.create_table(
            TableName=entityTableName,
            KeySchema=[
                {
                    'AttributeName': 'name',
                    'KeyType': 'HASH'
                },
                ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'name',
                    'AttributeType': 'S'
                },
                ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
                }
            )
        self._entities.meta.client.get_waiter('table_exists').wait(TableName=entityTableName)

        self._map = ddb.create_table(
            TableName=mapTableName,
            KeySchema=[
                {
                    'AttributeName': 'entity',
                    'KeyType': 'HASH'
                },
                {
                    'AttributeName': 'key',
                    'KeyType': 'RANGE'
                }
                ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'entity',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'key',
                    'AttributeType': 'S'
                },
                ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
                }
            )
        self._map.meta.client.get_waiter('table_exists').wait(TableName=mapTableName)
     
    def getEntity(self, name):
        return self._entities.get_item(Key={'name': name})
    def putEntity(self, item):
        self._entities.put_item(Item=item)
        
    def getMapEntries(self, entity):
        return self._map.query(KeyConditionExpression=Key('entity').eq(entity.meta.path()))
    def putMapEntry(self, item):
        self._map.put_item(Item=item)
        
    def _describe(self):
        return '%s: entities=%s, map=%s' % (self.name, self._entities.item_count, self._map.item_count)

In [1191]:
class TypeRegistry(object):
    def __init__(self):
        self.clsToName = {}
        self.nameToCls = {}
    def add(self, cls):
        """Testing method - just register an in-process class with its own name"""
        name = cls.__name__
        self.clsToName[cls] = name
        self.nameToCls[name] = cls
    def cls(self, name):
        return self.nameToCls[name]
    def name(self, cls):
        return self.clsToName[cls]
    
_tr = TypeRegistry()

In [1192]:
_uid = 10000
def getUUID():
    global _uid
    _uid = _uid+1
    s = str(_uid)
    return '%02d.%s' % (len(s), s)

In [1193]:
class Timestamp(object):
    def __init__(self, t=None, v=None):
        self.validTime       = v or datetime.datetime.utcnow()
        self.transactionTime = t or datetime.datetime.utcnow()
        assert self.validTime <= self.transactionTime
    def str(self):
        return self.transactionTime.isoformat()
    def writeForm(self):
        return [ self.transactionTime.isoformat(), self.validTime.isoformat() ]
    def __repr__(self):
        return '<TS:t=%s,v=%s>' % (self.transactionTime.isoformat(), self.validTime.isoformat())
    
    @classmethod
    def fromReadForm(cls, v):
        return Timestamp(dateutil.parser.parse(v[0]), 
                         dateutil.parser.parse(v[1]))

In [1194]:
class ObjectDbBase(object):
    _dbs = [None]
    
    def __init__(self):
        self.cache = {}
        
    def __enter__(self, *a):
        self._dbs.append(self)
    def __exit__(self, *a):
        db = self._dbs.pop()
        assert db == self

    def get(self, name):
        if name not in self.cache:
            self.cache[name] = self._get(name)
        return self.cache[name]
    
    def _allEventNames(self, entity):
        return [ i['event'] for i in self._allEventRecords(entity) ]
    
    def _allEvents(self, entity):
        names = self._allEventNames(entity)
        objs = [ self.get(name) for name in names ]
        return objs
    
    def describe(self):
        print '%s: %s' % (self, self._describe())
        
def currentDb():
    return ObjectDbBase._dbs[-1]

In [1195]:
class ObjectDb(ObjectDbBase):
    def __init__(self, dbDriver):
        self.dbDriver = dbDriver
        super(ObjectDb, self).__init__()
        self.name = 'O' + self.dbDriver.name
        
    def _reify(self, d, path, db):
        if 'Item' not in d:
            return None
        item = d['Item']
        cls = _tr.cls(item['type'])
        obj = cls(None, db=db)
        ts = item['timestamp']
        if ts:
            ts = Timestamp.fromReadForm(ts)
        obj._fromStoredForm(path, 
                            _payload   = item['payload'], 
                            _encoding  = item['encoding'], 
                            _timestamp = ts)
        return obj
    
    def _get(self, name, db=None):
        if db is None:
            db = self
        return self._reify(self.dbDriver.getEntity(name), name, db)
    
    def put(self, item):
        self.dbDriver.putEntity(item)
        
    def _allEventRecords(self, entity):
        response = self.dbDriver.getMapEntries(entity)
        return response['Items']
        
    def _putMapItem(self, item):
        self.dbDriver.putMapEntry(item)
        
    def _describe(self):
        return self.dbDriver._describe()
  
rawdb = DynamoDbDriver(ddb)
_odb = ObjectDb(rawdb)

In [1196]:
class UnionDb(ObjectDbBase):
    def __init__(self, frontDb, backDb):
        self.frontDb = frontDb
        self.backDb = backDb
        super(UnionDb, self).__init__()
        self.name = '(%s:%s)' %  (self.frontDb.name, self.backDb.name)
        
    def _get(self, name, db=None):
        # XXX - Really not adequate, would like caches on the child dbs,
        # we cache a modified copy
        if db is None:
            db = self
        if name not in self.cache:
            o = self.frontDb._get(name, db=db)
            if o is not None:
                return o
            return self.backDb._get(name, db=db)

    def put(self, item):
        self.frontDb.put(item)
        
    def _allEventRecords(self, entity):
        af = self.frontDb._allEventRecords(entity)
        ab = self.backDb._allEventRecords(entity)
        # XXX - want to merge here, not sort. But these will be iterators soon, so don't bother:
        return sorted(af + ab, key=lambda r: r['key'])
        
    def _putMapItem(self, item):
        self.frontDb._putMapItem(item)
        
    def _describe(self):
        return '(%s %s)' % (self.frontDb._describe(), self.backDb._describe())
  

In [1197]:
class EncDec(object):
    decoders = {}
    encoders = []
    
    def __init__(self, name, test, encode, decode):
        self.name = name
        self.test = test
        self.encode = encode
        self.decode = decode
        assert name not in self.decoders
        self.encoders.append(self)
        self.decoders[name] = decode
        
    @classmethod
    def encode(cls, value):
        for e in cls.encoders:
            if e.test(value):
                return e.name, e.encode(value)
        return None, value
    
    @classmethod
    def decode(cls, name, value, meta):
        if name is None:
            return value
        return cls.decoders[name](value, meta)
    
def encode(v):
    return EncDec.encode(v)

def decode(t, v, meta):
    return EncDec.decode(t, v, meta)
    
def addEncoding(name, test, encode, decode):
    EncDec(name, test, encode, decode)

def _persist(o):
    o.write()
    return o

addEncoding('O',
            lambda v: isinstance(v, DBO),
            lambda v: _persist(v).meta.path(),
            lambda v, meta: meta.db.get(v)
           )
                   
addEncoding('OL',
            lambda v: isinstance(v, list) and v,
            lambda v: [ _persist(o).meta.path() for o in v ],
            lambda v, meta: [ meta.db.get(p) for p in v ]
           )         

In [1198]:
class NoVal(object):
    pass

_noVal = NoVal()

In [1199]:
class Context(object):
    _contexts = []
    
    def __init__(self, tweaks):
        self.tweaks = tweaks
        self.cache = tweaks.copy()
    def __enter__(self, *a):
        self._contexts.append(self)
    def __exit__(self, *a):
        c = self._contexts.pop()
        assert c == self
    def get(self, cmb):
        return self.cache.get(cmb, _noVal)
    def set(self, cmb, v):
        self.cache[cmb] = v
  
    @classmethod
    def current(cls):
        if not cls._contexts:
            cls._contexts.append(Context({}))
        return cls._contexts[-1]
    
    @classmethod
    def inRootContext(cls):
        return cls.current() == cls._contexts[0]
    

In [1200]:
printWrites = False

class DBOMeta(object):
    def __init__(self, obj, name=None, db=None, kwargs=None):
        if db is None:
            db = currentDb()
        self.typeId       = _tr.name(obj.__class__)
        self._name        = name
        self.db           = db
        self._payload     = None
        self._encoding    = None
        self._timestamp   = None
        self._data        = kwargs
        self.isNew        = True
    def _fromStoredForm(self, path, _payload, _encoding, _timestamp):
        prefix = self._prefix()
        assert path.startswith(prefix)
        self._name = path[len(prefix):]
        op = {}
        while _encoding:
            v = _encoding.pop()
            k = _encoding.pop()
            op[k] = v
        self._payload   = _payload
        self._encoding  = op
        self._timestamp = _timestamp
        self.isNew      = False
    def __repr__(self):
        s = '<Meta %s:p=%s|en=%s|ts=%s|db=%s:%s>' % (self.path(),
                                                     self._payload, 
                                                     self._encoding, 
                                                     self._timestamp, 
                                                     self.db, 
                                                     self._data)
        return s
    def _prefix(self):
        return '/Global/%s/' % self.typeId
    def name(self):
        if self._name is None:
            self._name = getUUID()
        return self._name
    def path(self):
        return '%s%s' % (self._prefix(), self.name())
    def _toStoredForm(self):
        if self._encoding is None:
            enc = {}
            payload = {}
            for k, v in self._data.items():
                t, s = encode(v)
                payload[k] = s
                if t:
                    enc[k] = t
            self._payload = payload
            self._encoding = enc
            
    def getField(self, name):
        if self._payload and name in self._payload:
            p = self._payload[name]
            t = self._encoding.get(name)
            v = decode(t, p, self)
            return v
        elif self._data and name in self._data:
            return self._data[name]
        return _noVal
    
    def setField(self, name, value):
        assert self.isNew
        assert name not in self._data
        self._data[name] = value
        
    def _write(self, timestamp):
        if not Context.inRootContext():
            raise RuntimeError('non root-context write semantics not yet figured out.')
        path = self.path()
        self._toStoredForm() 
        if printWrites:
            print 'Writing (meta)', path, self.db.name
        op = []
        for k, v in self._encoding.items():
            op.append(k)
            op.append(v)
        item = {'name':      path,
                'type':      self.typeId,
                'payload':   self._payload,
                'encoding':  op,
                'timestamp': timestamp.writeForm() if timestamp else None,
                }
        db = self.db
        db.put(item)
            
    def write(self, timestamp=None, _containers=[]):
        if not self.isNew:
            return self
        self.isNew = False
        try:
            self._write(timestamp)
            path = self.path()
            return self
        except:
            self.isNew = True
            raise
            
class EventMeta(DBOMeta):
    def write(self, timestamp=None, _containers=[]):
        if not self.isNew:
            return self
        self.isNew = False
        try:
            if timestamp is None:
                timestamp = Timestamp()
            self._toStoredForm()
            db = self.db
            for v in _containers:
                m = _MapElement(v, self, timestamp, db)
                m.write()
            self._timestamp = timestamp
            self._write(timestamp)
        except:
            self.isNew = True
            raise

In [1201]:
def getValue(f, a, k):
    obj = a[0]
    name = f.func_name
    key = getattr(obj, name)
    ctx = Context.current()
    v = ctx.get(key)
    if v is not _noVal:
        return v
    v = obj.meta.getField(name)
    if v is not _noVal:
        return v
    v = f(*a, **k)
    if name in obj._storedFields():
        obj.meta.setField(name, v)
    ctx.set(key, v)
    
    return v

In [1202]:
def node(*a, **k):
    # XXX - this doesn't handle methods with args right
    if k:
        def g(*aa, **kk):
            for kw in k:
                assert kw in ('stored',)
            f = aa[0]
            info = k.copy()
            info['name'] = f.func_name
            def fn2(*aaa, **kkk):
                v = getValue(f, aaa, kkk)
                return v
            fn2.nodeInfo = info
            return fn2
        return g
    
    f = a[0]
    def fn(*aa, **kk):
        v = getValue(f, aa, kk)
        return v
    fn.nodeInfo = {'name': f.func_name}
    return fn

In [1203]:
class DBOMetaClass(type):
    def __new__(cls, name, parents, attrs):
        nodes = {}
        for parent in parents:
            if hasattr(parent, 'isDBO'):
                for nf in parent._nodes:
                    nodes[nf['name']] = nf
        for attrname, attrvalue in attrs.iteritems():
            if getattr(attrvalue, 'nodeInfo', 0):
                ni = attrvalue.nodeInfo
                nodes[ni['name']] = ni
                
        nodes = nodes.values()
        
        ret = super(DBOMetaClass, cls).__new__(cls, name, parents, attrs)
        ret._nodes = nodes
        
        return ret

In [1204]:
class DBO(object):
    __metaclass__ = DBOMetaClass
    _metaclass = DBOMeta
    isDBO = True
    
    @classmethod
    def _storedFields(cls):
        ret = []
        for n in cls._nodes:
            if n.get('stored'):
                ret.append(n['name'])
        return set(ret)
    
    def checkStoredFields(self, kwargs):
        stored = self._storedFields()
        bad = []
        for k in kwargs:
            if k not in stored:
                bad.append(k)
        if bad:
            raise RuntimeError('Setting non-stored fields: %s' % ', '.join(bad))
            
    def __init__(self, name=None, db=None, **kwargs):
        self.checkStoredFields(kwargs)
        self.meta = self._metaclass(self, name=name, db=db, kwargs=kwargs)
        
    def _fromStoredForm(self, path, _payload, _encoding, _timestamp):
        self.meta._fromStoredForm(path, _payload, _encoding, _timestamp)
        
    def write(self, _containers=[]):
        if self.meta.isNew:
            for n in self._storedFields():
                getattr(self, n)()
        self.meta.write(_containers=_containers)
        db = self.meta.db
        db.cache[self.meta.path()] = self
        return self
    
    def getField(self, name):
        return self.meta.getField(name)

    def _printDebug(self):
        print 'DEBUG for: %s: %s' % (self, str(self.meta))
        
class _MapElement(DBO):
    def __init__(self, entity, event, timestamp, db):
        self.entity = entity
        self.event = event
        self.timestamp = timestamp
        self.db = db
    def write(self):
        item = {'entity':    self.entity.meta.path(),
                'event':     self.event.path(),
                'key':       self.timestamp.str() + '|' + self.event.path(),
                }
        db = self.db
        db._putMapItem(item)
        return self

In [1205]:
class Entity(DBO):
    def _allEvents(self):
        db = self.meta.db
        return db._allEvents(self)
    
    def clock(self):
        db = self.meta.db
        return Clock.get('Main', db=db)
    
    def _visibleEvents(self):
        evs = self._allEvents()
        cutoffs = self.clock().cutoffs()
        if cutoffs:
            tTime = cutoffs.transactionTime
            evs = [ e for e in evs if e.meta._timestamp.transactionTime <= tTime ]
        return evs
    
    def activeEvents(self):
        evs = self._visibleEvents()
        cutoffs = self.clock().cutoffs()
        deletes = set()
        cancels = set()
        active = []
        for e in reversed(evs):
            if e in deletes:
                continue
            if isinstance(e, DeleteEvent):
                deletes.update(e._amends())
                continue
            if isinstance(e, CancelEvent):
                cancels.update(e._amends())
                continue
            else:
                deletes.update(e._amends())
            if e in cancels:
                continue
            active.append(e)

        if cutoffs:
            vTime = cutoffs.validTime
            active = [ e for e in active if e.meta._timestamp.validTime <= vTime ]
        return sorted(active, key=lambda e: e.meta._timestamp.validTime)
    
    def printActivity(self, evsFn=None):
        if evsFn is None:
            evsFn = self._visibleEvents
        print '%s %s:' % (self, evsFn.func_name)
        for e in evsFn():
            print '   ', e.str()
        print
       
    def str(self):
        return '<%s, isNew=%s>' % (self.meta.path(), self.meta.isNew)
    
    @classmethod
    def get(cls, name, db):
        typeId = _tr.name(cls)
        prefix = '/Global/%s/' % typeId
        path = '%s%s' % (prefix, name)
        return db.get(path)

    
class Event(DBO):
    _metaclass = EventMeta

    @node(stored=True)
    def amends(self):
        return []
    
    @node
    def _containers(self):
        ret = set()
        for a in self._amends():
            ret.update(a._containers())
        return ret
    
    def write(self):
        return super(Event, self).write(_containers=self._containers())
        
    def _amends(self):
        a = self.amends()
        if isinstance(a, list):
            return a
        if a:
            return [a]
        return []
    
    def str(self):
        return '<%s, isNew=%s, ts=%s>' % (self.meta.path(), self.meta.isNew, self.meta._timestamp)
    
    def delete(self):
        e = DeleteEvent(amends=self, db=self.meta.db)
        return e.write()
    
    def cancel(self):
        e = CancelEvent(amends=self, db=self.meta.db)
        return e.write()
    
class DeleteEvent(Event):
    pass

class CancelEvent(Event):
    pass

In [1206]:
class Clock(Entity):
    @node
    def cutoffs(self):
        return None

In [1207]:
_tr.add(Clock)
_tr.add(DeleteEvent)
_tr.add(CancelEvent)

In [1208]:
class RefData(Entity):

    def state(self):
        evs = self.activeEvents()
        cutoffs = self.clock().cutoffs()
        if evs:
            return evs[-1]
        
    def getField(self, name):
        s = self.state()
        if s:
            return s.getField(name)

class RefDataUpdateEvent(Event):
    
    @node(stored=True)
    def entity(self):
        return None
    
    def str(self):
        return '%s: %-15s: %-15s: %-30s' % (self.meta._timestamp.validTime,
                                            self.fullName(), 
                                            self.address(), 
                                            self.company(),
                                            ) 
    
    
    @node
    def _containers(self):
        ret = set()
        ret.update(super(RefDataUpdateEvent, self)._containers())
        ret.add(self.entity())
        return ret

_tr.add(RefData)
_tr.add(RefDataUpdateEvent)

In [1209]:
class CustomerRefData(RefData):
    
    def fullName(self):
        return self.state().fullName()
       
    def update(self, **kwargs):
        ev = CustomerRefDataUpdateEvent(entity=self, **kwargs)
        ev.write()
        return ev
    
class CustomerRefDataUpdateEvent(RefDataUpdateEvent):
    
    @node(stored=True)
    def fullName(self):
        return None
    
    @node(stored=True)
    def address(self):
        return None
    
    @node(stored=True)
    def company(self):
        return None
    
    @node(stored=True)
    def comment(self):
        return None
    
    
    def str(self):
        return '%s: %-15s: %-15s: %-30s: %s' % (self.meta._timestamp.validTime,
                                                self.getField('fullName'), 
                                                self.getField('address'), 
                                                self.getField('company'),
                                                self.entity().meta.name(),
                                                ) 

    
_tr.add(CustomerRefData)
_tr.add(CustomerRefDataUpdateEvent)

In [1210]:
_db = _odb
with _db:
    cl = Clock('Main').write()

    cr = CustomerRefData('Customer123')

    cr.update(fullName='Eliza Smith',
              address='10 Main St',
              company='Frobozz Magic Avocado Company')

    ts1 = Timestamp()

    e2 = cr.update(fullName='Eliza Smith',
                   address='10 Main St',
                   company='Frobozz Magic Friut Company')
    
    ts2 = Timestamp()

    cr.update(fullName='Eliza Smith',
              address='10 Main St',
              company='Frobozz Magic Fruit Company',
              comment='Grr. Typo.',
              amends = e2)

    ts3 = Timestamp()
    
    e4 = cr.update(fullName='Eliza James',
                   address='10 Main St',
                   company='Frobozz Magic Fruit Company')
    


cr.printActivity()

<__main__.CustomerRefData object at 0x107ec5090> _visibleEvents:
    2017-03-04 17:16:25.280516: Eliza Smith    : 10 Main St     : Frobozz Magic Avocado Company : Customer123
    2017-03-04 17:16:25.310826: Eliza Smith    : 10 Main St     : Frobozz Magic Friut Company   : Customer123
    2017-03-04 17:16:25.329564: Eliza Smith    : 10 Main St     : Frobozz Magic Fruit Company   : Customer123
    2017-03-04 17:16:25.352827: Eliza James    : 10 Main St     : Frobozz Magic Fruit Company   : Customer123



In [1211]:
print 'current:', cr.fullName()
with Context({cl.cutoffs: ts1}) as ctx:
    print 'at ts1 :', cr.fullName()
    print
    
    cr.printActivity(cr._visibleEvents)

cr.printActivity(cr._visibleEvents)
cr.printActivity(cr.activeEvents)

current: Eliza James
at ts1 : Eliza Smith

<__main__.CustomerRefData object at 0x107ec5090> _visibleEvents:
    2017-03-04 17:16:25.280516: Eliza Smith    : 10 Main St     : Frobozz Magic Avocado Company : Customer123

<__main__.CustomerRefData object at 0x107ec5090> _visibleEvents:
    2017-03-04 17:16:25.280516: Eliza Smith    : 10 Main St     : Frobozz Magic Avocado Company : Customer123
    2017-03-04 17:16:25.310826: Eliza Smith    : 10 Main St     : Frobozz Magic Friut Company   : Customer123
    2017-03-04 17:16:25.329564: Eliza Smith    : 10 Main St     : Frobozz Magic Fruit Company   : Customer123
    2017-03-04 17:16:25.352827: Eliza James    : 10 Main St     : Frobozz Magic Fruit Company   : Customer123

<__main__.CustomerRefData object at 0x107ec5090> activeEvents:
    2017-03-04 17:16:25.280516: Eliza Smith    : 10 Main St     : Frobozz Magic Avocado Company : Customer123
    2017-03-04 17:16:25.329564: Eliza Smith    : 10 Main St     : Frobozz Magic Fruit Company   : Cust

In [1212]:
_db.describe()

_db2 = ObjectDb(rawdb)
_db2.describe()
print

c = _db2.get(cr.meta.path())
assert c.clock() is not cl
print 'current:', c.fullName()
assert c.fullName() == 'Eliza James'

with Context({c.clock().cutoffs: ts1}) as ctx:
    print 'at ts1 :', c.fullName()
    assert c.fullName() == 'Eliza Smith'
    print
assert not c.meta.isNew

<__main__.ObjectDb object at 0x108686d50>: DDB-MEM-103: entities=6, map=4
<__main__.ObjectDb object at 0x108bde590>: DDB-MEM-103: entities=6, map=4

current: Eliza James
at ts1 : Eliza Smith



In [1213]:
_db3 = ObjectDb(DynamoDbDriver(ddb))
_db3.describe()

_dbU = UnionDb(_db3, _db)
_dbU.describe()

c = _dbU.get(cr.meta.path())
print 'current:', c.fullName()
with Context({c.clock().cutoffs: ts1}) as ctx:
    print 'at ts1 :', c.fullName()
    print
    
ev4 = _dbU.get(e4.meta.path())
print 'union event:', ev4, ev4.meta.db.name
ev4.delete()
c.printActivity()

_dbU.describe()
print

print 'in union:'
print 'current:', c.fullName()
with Context({c.clock().cutoffs: ts1}) as ctx:
    print 'at ts1 :', c.fullName()
    assert c.fullName() == 'Eliza Smith'
    print
    
print 'in production:'
_db2 = ObjectDb(rawdb)
_db2.describe()
print

c = _db2.get(cr.meta.path())
assert c.clock() is not cl
print 'current:', c.fullName()
assert c.fullName() == 'Eliza James'
with Context({c.clock().cutoffs: ts1}) as ctx:
    print 'at ts1 :', c.fullName()
    assert c.fullName() == 'Eliza Smith'
    print
assert not c.meta.isNew

<__main__.ObjectDb object at 0x108ae9790>: DDB-MEM-104: entities=0, map=0
<__main__.UnionDb object at 0x108ae9190>: (DDB-MEM-104: entities=0, map=0 DDB-MEM-103: entities=6, map=4)
current: Eliza James
at ts1 : Eliza Smith

union event: <__main__.CustomerRefDataUpdateEvent object at 0x108da3c10> (ODDB-MEM-104:ODDB-MEM-103)
<__main__.CustomerRefData object at 0x108c34bd0> _visibleEvents:
    2017-03-04 17:16:25.280516: Eliza Smith    : 10 Main St     : Frobozz Magic Avocado Company : Customer123
    2017-03-04 17:16:25.310826: Eliza Smith    : 10 Main St     : Frobozz Magic Friut Company   : Customer123
    2017-03-04 17:16:25.329564: Eliza Smith    : 10 Main St     : Frobozz Magic Fruit Company   : Customer123
    2017-03-04 17:16:25.352827: Eliza James    : 10 Main St     : Frobozz Magic Fruit Company   : Customer123
    </Global/DeleteEvent/05.10005, isNew=False, ts=<TS:t=2017-03-04T17:16:26.030125,v=2017-03-04T17:16:26.030116>>

<__main__.UnionDb object at 0x108ae9190>: (DDB-MEM-104:

In [1214]:
def merge(d, d2, deleteZeros=True):
    for k, v in d2.items():
        d[k] = d.get(k, 0) + v
        if deleteZeros and d[k] == 0:
            del d[k]
        
class Workbook(Entity):
    
    def workItems(self):
        evs = self.activeEvents()
        ret = {}
        for e in evs:
            ibb = e._itemsByBook()
            items = ibb.get(self, {})
            merge(ret, items)
        return ret
        
    def printOpenItems(self):
        for k in self.workItems():
            print k.str()
            
    def printStatus(self):
        print 'activity:'
        self.printActivity()
        print 'open items:'
        self.printOpenItems()
        
class WorkItem(Entity):
    
    @node
    def openEvent(self):
        evs = self.activeEvents()
        op = [ e for e in evs if isinstance(e, WorkItemOpenEvent) ]
        if len(op) > 1:
            assert False
        return op[0] if op else None

    @node
    def _containers(self):
        oe = self.openEvent()
        if oe:
            return oe._containers()
        
    def addMessage(self, message):
        ev = WorkItemMessageEvent(message=message, item=self, db=self.meta.db)
        ev.write()
        return ev
    
class WorkItemOpenEvent(Event):

    @node(stored=True)
    def item(self):
        ret = WorkItem(db=self.meta.db)
        return ret
    
    @node(stored=True)
    def book1(self):
        return None
    
    @node(stored=True)
    def book2(self):
        return None
    
    @node(stored=True)
    def message(self):
        return None
    
    def ticket(self):
        return self.item()
    
    def _items(self):
        return [ [ self.item(), 1, self.book1(), self.book2() ] ]
    
    def _itemsByBook(self):
        """returns { book: {item: q } }"""
        ret = {}
        def add(i, q, b):
            if b not in ret:
                ret[b] = {}
            if i not in ret[b]:
                ret[b][i] = 0
            ret[b][i] += q
        for i, q, b1, b2 in self._items():
            add(i, q, b1)
            add(i, -q, b2)
        return ret
    
    def str(self):
        return 'New, %s: %s %s' % (self.meta.name(), self.book1().meta.name(), self.message())
    
    @node
    def _containers(self):
        ret = set()
        ret.update(super(WorkItemOpenEvent, self)._containers())
        ret.add(self.book1())
        ret.add(self.book2())
        ret.add(self.item())
        return ret

class WorkItemMessageEvent(Event):
    
    @node(stored=True)
    def item(self):
        return None
    
    @node(stored=True)
    def message(self):
        return None
    
    def _itemsByBook(self):
        return {}
        
    @node
    def _containers(self):
        ret = set()
        ret.update(super(WorkItemMessageEvent, self)._containers())
        ret.add(self.item())
        c = self.item()._containers()
        ret.update(c)
        return ret
    
    def str(self):
        return 'Message, Opened by: %s' % (self.item().openEvent())


In [1215]:
_tr.add(Workbook)
_tr.add(WorkItem)
_tr.add(WorkItemOpenEvent)
_tr.add(WorkItemMessageEvent)

In [1216]:
with _db:
    wb1 = Workbook('Customer123')
    hd  = Workbook('Helpdesk')
    wb3 = Workbook('Customer.joe')

    startOfDay = Timestamp()
    
    ev1 = WorkItemOpenEvent(message='Help, I forgot my password',
                            book1=wb1,
                            book2=hd).write()
    ev2 = WorkItemOpenEvent(message='Help! My computer is on fire!',
                            book1=wb3,
                            book2=hd).write()
    
    noon = Timestamp()
    
    ev3 = WorkItemOpenEvent(message='My mouse is broken',
                            book1=wb1,
                            book2=hd).write()
    
    ticket=ev3.ticket()
    
    #ev3._printDebug()
    
    ticket.addMessage('Actually, only the right mouse button is bad, so just replace that. Thx!')
    
    t3 = Timestamp()
    
    ev1.cancel()
    
    endOfDay = Timestamp()

In [1217]:
with Context({cl.cutoffs: startOfDay}) as ctx:
    hd.printStatus()

activity:
<__main__.Workbook object at 0x108c34290> _visibleEvents:

open items:


In [1218]:
with Context({cl.cutoffs: noon}) as ctx:
    hd.printStatus()

activity:
<__main__.Workbook object at 0x108c34290> _visibleEvents:
    New, 05.10007: Customer123 Help, I forgot my password
    New, 05.10009: Customer.joe Help! My computer is on fire!

open items:
</Global/WorkItem/05.10006, isNew=False>
</Global/WorkItem/05.10008, isNew=False>


In [1219]:
with Context({cl.cutoffs: t3}) as ctx:
    hd.printStatus()

activity:
<__main__.Workbook object at 0x108c34290> _visibleEvents:
    New, 05.10007: Customer123 Help, I forgot my password
    New, 05.10009: Customer.joe Help! My computer is on fire!
    New, 05.10011: Customer123 My mouse is broken
    Message, Opened by: <__main__.WorkItemOpenEvent object at 0x108a370d0>

open items:
</Global/WorkItem/05.10006, isNew=False>
</Global/WorkItem/05.10010, isNew=False>
</Global/WorkItem/05.10008, isNew=False>


In [1220]:
with Context({cl.cutoffs: endOfDay}) as ctx:
    hd.printStatus()

activity:
<__main__.Workbook object at 0x108c34290> _visibleEvents:
    New, 05.10007: Customer123 Help, I forgot my password
    New, 05.10009: Customer.joe Help! My computer is on fire!
    New, 05.10011: Customer123 My mouse is broken
    Message, Opened by: <__main__.WorkItemOpenEvent object at 0x108a370d0>
    </Global/CancelEvent/05.10013, isNew=False, ts=<TS:t=2017-03-04T17:16:26.960155,v=2017-03-04T17:16:26.960151>>

open items:
</Global/WorkItem/05.10008, isNew=False>
</Global/WorkItem/05.10010, isNew=False>


In [1221]:
item = ev1.item()

def prn():
    print item
    print '  Open   :', item.openEvent()
    print '  Active :', item.activeEvents()

prn()
print
print 'At noon:'
with Context({cl.cutoffs: noon}) as ctx:
    prn()

  Open   : None
  Active : []

At noon:
  Open   : <__main__.WorkItemOpenEvent object at 0x108592ad0>
  Active : [<__main__.WorkItemOpenEvent object at 0x108592ad0>]
